In [2]:

import gc
import polars as pl
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import seaborn as sns
import pandas as pd
from collections import Counter
from heapq import nlargest





In [7]:
df_pol_train = pl.read_parquet('../data/train.parquet')
df_pol_test = pl.read_parquet('../data/test.parquet')

In [ ]:
df_test_json = pd.read_json("../data/test.jsonl", lines=True)

In [3]:
df_train_json = pd.read_json("../data/train.jsonl", lines=True, chunksize = 100000)

In [4]:
df_train_json

In [5]:

df = pd.DataFrame.from_dict(df_train_json, orient="index")

TypeError: object of type 'JsonReader' has no len()

In [31]:
i = 0
for row in df_pol_train["type"]:
   
   for session in row:
 
         if value == "clicks":
            sample_test_df["test_clicks"][i].append(session["aid"])
         if value == "carts": 
            sample_test_df["test_carts"][i].append(session["aid"])
         if value == "orders":
            sample_test_df["test_orders"][i].append(session["aid"])
      
   i += 1

type
u8
0


Lets find the most common article for each different type of action

In [37]:

#sample_df = df_test_json[:150000]
clicks_article_list = []
carts_article_list = []
orders_article_list = []
    
i = 0    
for type in df_pol_train["type"]:
        if type == 0:
                clicks_article_list.append(df_pol_train["aid"][i])
        elif type == 1:
                carts_article_list.append(df_pol_train["aid"][i])
        else:
                orders_article_list.append(df_pol_train["aid"][i])

        i += 1

In [38]:
# Create dictionaries with articles and their frequencies
article_click_freq = Counter(clicks_article_list)
article_carts_freq = Counter(carts_article_list)
article_order_freq = Counter(orders_article_list)

In [39]:
top_click_article = sorted(article_click_freq, key=article_click_freq.get, reverse=True)[:20]
top_carts_article = sorted(article_carts_freq, key=article_carts_freq.get, reverse=True)[:20]
top_order_article = sorted(article_order_freq, key=article_order_freq.get, reverse=True)[:20]

In [52]:
sorted(article_click_freq, key=article_click_freq.get, reverse=True)[:20]

[1460571,
 108125,
 29735,
 485256,
 1733943,
 184976,
 832192,
 1502122,
 554660,
 1603001,
 986164,
 166037,
 322370,
 1236775,
 231487,
 959208,
 332654,
 1196256,
 95488,
 620545]

In [ ]:
# Create a dict with this info
frequent_articles = {'clicks': top_click_article, 'carts':top_carts_article, 'order':top_order_article}

In [ ]:
for action in ['clicks', 'carts', 'order']:
    print(f'Most frequent articles for {action}: {frequent_articles[action][:5]}')

y_test = week nr 5 
y_train = weeks 1-4

y= each step in a session after truncation by timestamp (not done yet by the costumer in real session)
x= every step in the session, before truncation by timestamp. (all the clicks, etc. that the costumer has done in a session)

in order to measure if our baseline module are predicting correctly, one of the recommendations should have been clicked, added to chart or ordered in the y_test.

For each session in the test data, your task it to predict the aid values for each type that occur after the last timestamp ts the test session. In other words, the test data contains sessions truncated by timestamp, and you are to predict what occurs after the point of truncation.

In [ ]:
df_train

In [ ]:
sample_df

In [ ]:
"""
# To do
# create another loop to go through the rows
# inside the loop, for every session: calculate the recall and add it in a new column to the row

#idea: get the length of the session (number of dicts) and then only chose the last 50% for test data

# start with a loop, three empty lists, each session will be filled into the three lists, save the results into new columns, then empty the lists for the next iteration

# question: for the whole test data, is there a different way than looping?

# Creating test data
test_clicks = []
test_carts = []
test_orders = []

# looping through one session (JSON-format) and appending the aid to lists for each type
for dictionary in sample_df["events"][14]:
    data = dictionary
    for key, value in data.items():
     if value == "clicks":
        test_clicks.append(data["aid"])

     if value == "carts":
        test_carts.append(data["aid"])   
     if value == "orders":
        test_orders.append(data["aid"])
print(test_clicks, test_orders)
"""

In [ ]:
# To do
# create another loop to go through the rows
# inside the loop, for every session: calculate the recall and add it in a new column to the row

#idea: get the length of the session (number of dicts) and then only chose the last 50% for test data

# start with a loop, three empty lists, each session will be filled into the three lists, save the results into new columns, then empty the lists for the next iteration

# question: for the whole test data, is there a different way than looping?

# create a new dataframe and add columns for the test data
sample_test_df = sample_df.copy()

sample_test_df["test_clicks"] = [[] for i in range(len(sample_test_df))]
sample_test_df["test_carts"] = [[] for i in range(len(sample_test_df))]
sample_test_df["test_orders"] = [[] for i in range(len(sample_test_df))]


# loop through the dataframe (150 sessions).
# starting from the second half of a session (JSON format) : grab each event, check the type 
# and put it into the according column of test-data.

i = 0
for row in sample_test_df["events"]:
   length = len(row)
   for session in row[int(length/2):]:
      for key, value in session.items():
         if value == "clicks":
            sample_test_df["test_clicks"][i].append(session["aid"])
         if value == "carts": 
            sample_test_df["test_carts"][i].append(session["aid"])
         if value == "orders":
            sample_test_df["test_orders"][i].append(session["aid"])
      
   i += 1




In [ ]:
sample_test_df

In [ ]:
# modelling:
# idea: for the first product, start with most popular products as recommendations
# then take each product as an input for predicting the next one
# for the first 3 (?) products, mix most popular and related recommendations
# after that give only related recommendations
# try different versions (different mix of recommendations)

In [ ]:
#calculate recall
# this function takes six lists to calculate the recall for one session.
# three lists with predicted aid and three lists with true aid.
# to calculate the recall for each type, we will check the intersection of two sets, 
# predicted aid and true aid. If one of our predictions is true, the recall will be one.
r_total = []

def calc_recall(pred_clicks, pred_carts, pred_orders, test_clicks, test_carts, test_orders):
    # first calculate the recall for each type of prediction
    if set(pred_clicks).intersection(set(test_clicks)):
        r_click = 1
    else:
        r_click = 0
    if set(pred_carts).intersection(set(test_carts)):
        r_carts = 1
    else:
        r_carts = 0
    if set(pred_orders).intersection(set(test_orders)):
        r_order = 1
    else:
        r_order = 0
    
    # returning the total recall as a weighted sum of the individual recalls
    r_session = 0.1*r_click + 0.3*r_carts + 0.6*r_order

    r_total.append(r_session)
    


In [40]:
# Calculate the recall for every session, using the top 20 aid of this type as predictions.
# the calc_recall function calculates the weighted sum of recall for a session and then appends it to the list r_total

for j in range(len(sample_test_df)):
    calc_recall(top_click_article, top_carts_article, top_order_article, sample_test_df["test_clicks"][j],sample_test_df["test_carts"][j],sample_test_df["test_orders"][j])
    
# the final result is the sum of recalls in r_total divided by the length of r_total   
result = sum(r_total)/len(r_total)
print(result)

NameError: name 'sample_test_df' is not defined

In [42]:
submission = pd.read_csv("../data/sample_submission.csv")

In [45]:
submission.head()

,session_type,labels
0,12899779_clicks,129004 126836 118524
1,12899779_carts,129004 126836 118524
2,12899779_orders,129004 126836 118524
3,12899780_clicks,129004 126836 118524
4,12899780_carts,129004 126836 118524


In [50]:
submission['labels'] = submission['labels'].apply(lambda x: top_click_article )


In [51]:
submission.head()

,session_type,labels
0,12899779_clicks,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
1,12899779_carts,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
2,12899779_orders,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
3,12899780_clicks,"[1460571, 108125, 29735, 485256, 1733943, 1849..."
4,12899780_carts,"[1460571, 108125, 29735, 485256, 1733943, 1849..."


In [66]:
top_clicks_submission = ' '.join(str(i) for i in top_click_article)
top_carts_submission = ' '.join(str(i) for i in top_carts_article)
top_order_submission = ' '.join(str(i) for i in top_order_article)


In [67]:
def check_type(x):
    if "click" in x:
        return top_clicks_submission
    elif "carts" in x:
        return top_carts_submission
    else:
        return top_order_submission

In [68]:
submission['labels'] = submission['session_type'].apply(check_type )

In [69]:
submission.head()

,session_type,labels
0,12899779_clicks,1460571 108125 29735 485256 1733943 184976 832...
1,12899779_carts,485256 152547 33343 166037 1733943 231487 2973...
2,12899779_orders,231487 166037 1733943 1445562 1022566 801774 1...
3,12899780_clicks,1460571 108125 29735 485256 1733943 184976 832...
4,12899780_carts,485256 152547 33343 166037 1733943 231487 2973...


In [70]:
submission.to_csv('submission.csv', index=False)